In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
data = np.load("C:\\Users\\ShwetaSood\\Desktop\\Bioinformatics\\dataset_abundance_obe.npy")
y_data = pd.read_csv("C:\\Users\\ShwetaSood\\Desktop\\Bioinformatics\\obephy_y.csv")
indices = np.random.permutation(252)
y_data = y_data.iloc[:,1]
y_data = tf.one_hot(y_data, depth=2, axis=-1)
sess1 = tf.Session()

y_data = sess1.run(y_data)

xtrain = data[indices[0:177]]
ytrain = y_data[indices[0:177]]

xtest = data[indices[177:252]]
ytest = y_data[indices[177:252]]


xlimit = 465

In [2]:
xtrain.shape

(177, 465)

In [18]:
optimal_indices = indices

In [10]:
tf.reset_default_graph()
################################ Neural Network Unit ###############################################
def distance(p1,p2):
    return abs(p1-p2)
nn_num = 0
def one_layer(X, n_input, n_output, activation=None, scope=None):
    with tf.variable_scope(scope or "linear"):
        W = tf.get_variable(
            name='W',
            shape=[n_input, n_output],
            initializer=tf.random_normal_initializer(mean=0, stddev=0.1))        
        h = tf.matmul(X, W)
        if activation is not None:
            h = activation(h)
        return h
    
X = tf.placeholder(tf.float32, shape=[None, xlimit], name='X')
Y = tf.placeholder(tf.float32, shape=[None,2], name='Y')
n_neurons = [xlimit,10,2]
current_input = X
with tf.Session() as sess:
    for layer_i in range(1, len(n_neurons)):
        current_input = one_layer(
            X=current_input,
            n_input=n_neurons[layer_i - 1],
            n_output=n_neurons[layer_i],
            activation=tf.nn.relu if (layer_i+1) < len(n_neurons) else tf.nn.softmax,
            scope='layer_' + str(layer_i)) + nn_num
    Y_pred = current_input   
    g=tf.get_default_graph()
    cross_entropy = -tf.reduce_sum(Y * tf.log(Y_pred + 1e-12))
    optimizer = tf.train.AdamOptimizer(0.01).minimize(cross_entropy) 
    sess.run(tf.global_variables_initializer())
    correct_prediction = tf.equal(tf.argmax(Y_pred, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    
    

In [23]:
################################## Train & Test Phase ##################################################
niter=10
sess = tf.Session()
sess.run(tf.global_variables_initializer())
batch_size = 70

for i in range(niter) :
    for i in range(0,177,batch_size):
            if(i+batch_size<len(xtrain)):
                xbatch = xtrain[i:i+batch_size]
                ybatch = ytrain[i:i+batch_size]
            else:
                xbatch = xtrain[i:len(xtrain)]
                ybatch = ytrain[i:len(xtrain)]

            sess.run(optimizer,feed_dict={X : xbatch ,Y : ybatch })
            print("Iteration" + str(i))
            print("Train Accuracy : ")    
            print(sess.run(accuracy,
                           feed_dict={
                               X: xtrain,
                               Y: ytrain
                           }))

acc_0 = sess.run(accuracy,
                   feed_dict={
                       X: xtest,
                       Y: ytest
                   })
tr_e =sess.run(correct_prediction, feed_dict = {X : xtrain, Y : ytrain})
ts_e =sess.run(correct_prediction, feed_dict = {X : xtest, Y : ytest})
Y_pred0 = sess.run(Y_pred, feed_dict = {X : xtest})


e_i = []
c_i =[]
for i in range(len(tr_e)):
    if tr_e[i]==False:
        e_i.append(i)
    else:
        c_i.append(i)

xtrain2 = xtrain[e_i]
ytrain2 = ytrain[e_i]
nn_num=1
for i in range(niter) :
    for i in range(0,177,batch_size):
            if(i+batch_size<len(xtrain)):
                xbatch = xtrain2[i:i+batch_size]
                ybatch = ytrain2[i:i+batch_size]
            else:
                xbatch = xtrain2[i:len(xtrain)]
                ybatch = ytrain2[i:len(xtrain)]
            sess.run(optimizer,feed_dict={X : xbatch ,Y : ybatch })
            print("Iteration" + str(i))
            print("Test Accuracy : ")    
            print(sess.run(accuracy,
                           feed_dict={
                               X: xtest,
                               Y: ytest
                           }))
            print("Train Accuracy : ")    
            print(sess.run(accuracy,
                           feed_dict={
                               X: xtrain2,
                               Y: ytrain2
                           }))

acc_1 = sess.run(accuracy,
                   feed_dict={
                       X: xtest,
                       Y: ytest
                   })
tr_e2 =sess.run(correct_prediction, feed_dict = {X : xtrain2, Y : ytrain2})
ts_e2 =sess.run(correct_prediction, feed_dict = {X : xtest, Y : ytest})

esi_test=[]
csi_test =[]

for i in range(len(ts_e)):
    if ts_e[i]==False:
        esi_test.append(i)
    else :
        csi_test.append(i)

esi2_test=[]
for i in range(len(ts_e2)):
    if ts_e2[i]==False:
        esi2_test.append(i)


Y_pred1 = sess.run(Y_pred, feed_dict = {X : xtest})

'''
for i in esi_test:
    print("Test example " + str(i))
    print(Y_pred0[i])
    print("Compared with")
    print(Y_pred1[i])
'''  
delta = np.arange(0.2,0.3,0.05)
for th in delta : 
    t=0

    Y_preda1 = sess.run(tf.equal(tf.argmax(Y_pred, 1), tf.argmax(Y, 1)), feed_dict = {X : xtest, Y : ytest})
    for i in csi_test:
        if(np.amax(Y_pred0[i])+th <np.amax(Y_pred1[i])):
            if(Y_preda1[i]==False) :
                t=t+1


    for i in esi_test:
        if(np.amax(Y_pred0[i])+th >np.amax(Y_pred1[i])):
            t=t+1

    for i in esi_test:
        if(np.amax(Y_pred0[i])+th <np.amax(Y_pred1[i])):
            if(Y_preda1[i]==False) :
                t=t+1
    print("Number of errors " + str(t) + "for Theta value" + str(th))
    print("Accuracy="+str((75-t)/75))





Iteration0
Train Accuracy : 
0.43502825
Iteration70
Train Accuracy : 
0.6836158
Iteration140
Train Accuracy : 
0.68926555
Iteration0
Train Accuracy : 
0.65536726
Iteration70
Train Accuracy : 
0.6440678
Iteration140
Train Accuracy : 
0.6384181
Iteration0
Train Accuracy : 
0.6384181
Iteration70
Train Accuracy : 
0.63276833
Iteration140
Train Accuracy : 
0.62711865
Iteration0
Train Accuracy : 
0.62711865
Iteration70
Train Accuracy : 
0.62711865
Iteration140
Train Accuracy : 
0.6214689
Iteration0
Train Accuracy : 
0.6214689
Iteration70
Train Accuracy : 
0.6214689
Iteration140
Train Accuracy : 
0.6214689
Iteration0
Train Accuracy : 
0.6214689
Iteration70
Train Accuracy : 
0.6214689
Iteration140
Train Accuracy : 
0.6214689
Iteration0
Train Accuracy : 
0.6214689
Iteration70
Train Accuracy : 
0.6214689
Iteration140
Train Accuracy : 
0.6214689
Iteration0
Train Accuracy : 
0.6214689
Iteration70
Train Accuracy : 
0.6214689
Iteration140
Train Accuracy : 
0.6214689
Iteration0
Train Accuracy : 
0.62